They say that education is important, but beer is importatanter. Some people say thet education is important, but big biceps are importatanter. What does bert think about them? I tried masked language model.
<br/>
If you never heard of these quotes, check this URL.
https://www.google.com/search?q=education+is+important+but+beer+is+importanter&source=lnms&tbm=isch&sa=X&ved=2ahUKEwjn1snlisj2AhX1zIsBHWQwBhYQ_AUoAnoECAIQBA&biw=1366&bih=657&dpr=1

<br/>
<br/>
I used bert-large-cased for pretrained model.
<br/>
https://huggingface.co/transformers/v3.3.1/pretrained_models.html

#0 Preparation

In [5]:
!git clone https://github.com/google-research/bert
!pip install transformers==4.5.0

Cloning into 'bert'...
remote: Enumerating objects: 340, done.
remote: Total 340 (delta 0), reused 0 (delta 0), pack-reused 340
Receiving objects: 100% (340/340), 328.28 KiB | 4.16 MiB/s, done.
Resolving deltas: 100% (182/182), done.
     |████████████████████████████████| 2.1 MB 5.1 MB/s 
     |████████████████████████████████| 895 kB 38.2 MB/s 
     |████████████████████████████████| 3.3 MB 9.2 MB/s 


In [6]:
import numpy as np
import torch
from transformers import BertModel, BertForMaskedLM,BertTokenizer

In [8]:
model_name = "bert-large-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
bert_mlm = BertForMaskedLM.from_pretrained(model_name) #mlm stands for masked language model
bert_mlm = bert_mlm.cuda()

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
def predict_mask_topk(text, tokenizer, bert_mlm, num_topk):
    """
    replace [MASK] with high socre token(s)in the text. 
    num_topk: the number of predictions to show
    The outputs show the sentence(s) and token score(s) 
    """
    # encode the sentence and get the scores in BERT
    input_ids = tokenizer.encode(text, return_tensors='pt')
    input_ids = input_ids.cuda()
    with torch.no_grad():
        output = bert_mlm(input_ids=input_ids)
    scores = output.logits

    # Get top tokens
    mask_position = input_ids[0].tolist().index(103) #ID of [MASK] is 103
    topk = scores[0, mask_position].topk(num_topk)
    ids_topk = topk.indices # token ID
    tokens_topk = tokenizer.convert_ids_to_tokens(ids_topk) # token
    scores_topk = topk.values.cpu().numpy() # score

    # replace MASK with the token from model。
    text_topk = [] # add filled texts
    for token in tokens_topk:
        token = token.replace('##', '')
        text_topk.append(text.replace('[MASK]', token, 1))

    return text_topk, scores_topk



#1 "Education is important, but (_) is importanter."

I replaced beer with [MASK].
<br/>
Does BERT think beer is importanter?


In [ ]:
text1 = "Education is important, but [MASK] is importanter."

In [ ]:
text_topk, _ = predict_mask_topk(text1, tokenizer, bert_mlm, 10)
text_topk

['Education is important, but money is importanter.',
 'Education is important, but health is importanter.',
 'Education is important, but income is importanter.',
 'Education is important, but religion is importanter.',
 'Education is important, but culture is importanter.',
 'Education is important, but business is importanter.',
 'Education is important, but work is importanter.',
 'Education is important, but family is importanter.',
 'Education is important, but employment is importanter.',
 'Education is important, but wealth is importanter.']

Nothing looks weired. <br/>
I guess religion wouldn't be importanter in Japanese pretrained model.




#2. Education is important, but (_) is more important.

what if impotanter is more important?

In [7]:
text2 = "Education is important, but [MASK] is more important."
text_topk2, _ = predict_mask_topk(text2, tokenizer, bert_mlm, 10)
text_topk2

['Education is important, but family is more important.',
 'Education is important, but money is more important.',
 'Education is important, but business is more important.',
 'Education is important, but health is more important.',
 'Education is important, but work is more important.',
 'Education is important, but knowledge is more important.',
 'Education is important, but religion is more important.',
 'Education is important, but education is more important.',
 'Education is important, but income is more important.',
 'Education is important, but culture is more important.']

They are similar to 1, but not the same.

#3. (_) is important, but beer is importanter

Does BERT think education is important?

In [10]:
text3 = "[MASK] is important, but beer is importanter."
text_topk3, _ = predict_mask_topk(text3, tokenizer, bert_mlm, 10)
text_topk3

['Food is important, but beer is importanter.',
 'Wine is important, but beer is importanter.',
 'Water is important, but beer is importanter.',
 'Meat is important, but beer is importanter.',
 'Beer is important, but beer is importanter.',
 'Oil is important, but beer is importanter.',
 'Coffee is important, but beer is importanter.',
 'Milk is important, but beer is importanter.',
 'Tea is important, but beer is importanter.',
 'Fish is important, but beer is importanter.']

All predictions are related to food or something to drink.
<br/>
5th sentense is weired. It says beer is importanter than beer.


#4. Education is important, but big (_) are importanter.

Big biceps are importanter?

In [9]:
text4 = "Education is important, but big [MASK] are importanter."
text_topk4, _ = predict_mask_topk(text4, tokenizer, bert_mlm, 10)
text_topk4

['Education is important, but big businesses are importanter.',
 'Education is important, but big companies are importanter.',
 'Education is important, but big cities are importanter.',
 'Education is important, but big business are importanter.',
 'Education is important, but big industries are importanter.',
 'Education is important, but big jobs are importanter.',
 'Education is important, but big corporations are importanter.',
 'Education is important, but big numbers are importanter.',
 'Education is important, but big things are importanter.',
 'Education is important, but big families are importanter.']

Mostly related to work

#5 Education is important, but (_) is not important.

In [13]:
text5 = "Education is important, but [MASK] is not important."
text_topk5, _ = predict_mask_topk(text5, tokenizer, bert_mlm, 10)
text_topk5

['Education is important, but money is not important.',
 'Education is important, but income is not important.',
 'Education is important, but religion is not important.',
 'Education is important, but technology is not important.',
 'Education is important, but education is not important.',
 'Education is important, but wealth is not important.',
 'Education is important, but sex is not important.',
 'Education is important, but knowledge is not important.',
 'Education is important, but work is not important.',
 'Education is important, but culture is not important.']

It looks almost the same as the sentence without not.
<br>
They are all important.
<br/>